In [1]:
import torch
import numpy as np 
print(torch.__version__)
# 台式机是pytorch1，有cuda

2.2.1


# 第一个-全连接神经网络-判断矩阵“胖瘦”即高宽

In [25]:
torch.rand(1)

tensor([0.2420])

In [56]:
def get_rectangle():
    width = torch.rand(1)
    height = torch.rand(1)
    fat = int(height < width)
    return width,height,fat
get_rectangle()

(tensor([0.2957]), tensor([0.9372]), 0)

In [291]:
#1.定义数据集
# 注意大小写
class DataSet(torch.utils.data.Dataset):
    def __init__(self):
        pass
    def __len__(self):
        return 500
    def __getitem__(self,index):
        width,height,fat = get_rectangle()
        # x 是一维的   有两个数据
        X = torch.FloatTensor([width,height])
        # X是tensor ， 但是y不是
        Y = fat
        return X,Y
dataset = DataSet()
len(dataset),dataset[1000]


(500, (tensor([0.9892, 0.2312]), 1))

In [300]:
# 2.数据集加载器
# 有些英语单词是连着的，有些英语单词又用 _ 分割
loader = torch.utils.data.DataLoader(dataset = dataset,batch_size = 8, shuffle =True, drop_last = True)
# loader自动会将数据变成tensor 
len(loader),next(iter(loader))[0].shape,next(iter(loader))[1].shape

(62, torch.Size([8, 2]), torch.Size([8]))

In [306]:
# 3.全连接神经网络 分类任务
class Model(torch.nn.Module):
    def __init__(self):
        #一定要写继承
        super().__init__()

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(2,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,2),
            #softmax  
            torch.nn.Softmax(dim=1)
        )
    def forward(self,x):
        return self.fc(x)
    
model = Model()

model(torch.randn(8,2)).shape

torch.Size([8, 2])

In [346]:
# 分析一下
model = Model()
def get_shape(module, input, output):
    print(f"Input shape: {input[0].shape}")
    print(f"Output shape: {output.shape}")
# 注册forward hook来获取每一层的输入和输出形状变化
for name, layer in model.named_children():
    #print(name,layer)
    #sequential 默认是一层
    layer.register_forward_hook(get_shape)

# 开始打印每一层的变化
model(torch.randn(8,2))

Input shape: torch.Size([8, 2])
Output shape: torch.Size([8, 2])


tensor([[0.4976, 0.5024],
        [0.4993, 0.5007],
        [0.5131, 0.4869],
        [0.4665, 0.5335],
        [0.4769, 0.5231],
        [0.4901, 0.5099],
        [0.5049, 0.4951],
        [0.5149, 0.4851]], grad_fn=<SoftmaxBackward>)

In [351]:
# 4.训练
model = Model()
def train():
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    loss_fun = torch.nn.CrossEntropyLoss()

    model.train()
    for epoch in range(100):
        for i,(x,y) in enumerate(loader):
            out = model(x)
           
            loss = loss_fun(out,y)

            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if epoch%20 == 0:
            #都是训练集的结果，其实acc和loss.item()代表的含义差不多
            acc = (out.argmax(dim=1) == y).sum().item() / len(y)
            #item只能取一个元素的
            print(epoch,loss.item(),acc)
    torch.save(model,'./test1_model')

#开始训练
train()

0 0.7242922782897949 0.25
20 0.5402665734291077 1.0
40 0.42121559381484985 0.875
60 0.3642852306365967 1.0
80 0.31795355677604675 1.0


In [387]:
# 5.测试
@torch.no_grad()
def test():
    
    #先加载模型，当然要是从头到尾跑的那也可以
    model = torch.load('./test1_model')

    #进入测试模式
    model.eval()

    #获取一批数据
    x,y = next(iter(loader))
    out = model(x)
    print(out)
    print(y)
    print(out.argmax(dim=1) == y)
    criterion = torch.nn.CrossEntropyLoss()
    loss = criterion(out, y)
    print(loss.item())
    #很奇怪，下面这样连这写是错误的
    #print(torch.nn.CrossEntropyLoss(out,y).item())
test()

tensor([[9.9907e-01, 9.2597e-04],
        [9.9940e-01, 6.0503e-04],
        [9.9912e-01, 8.7973e-04],
        [7.0180e-05, 9.9993e-01],
        [9.9819e-01, 1.8142e-03],
        [9.9964e-01, 3.5509e-04],
        [1.6649e-03, 9.9834e-01],
        [9.9937e-01, 6.3114e-04]])
tensor([0, 0, 0, 1, 0, 0, 1, 0])
tensor([True, True, True, True, True, True, True, True])
0.3137291371822357


# 第二个-计算矩阵的面积

In [2]:
def get_rectangle():
    width = torch.rand(1)
    height = torch.rand(1)
    s = width * height
    return width,height,s
get_rectangle()

(tensor([0.5198]), tensor([0.4294]), tensor([0.2232]))

In [13]:
#1.定义数据集
# 注意大小写
class DataSet(torch.utils.data.Dataset):
    def __init__(self):
        pass
    def __len__(self):
        return 500
    def __getitem__(self,index):
        width,height,s = get_rectangle()
        # x 是一维的   有两个数据
        X = torch.FloatTensor([width,height])
        # X是tensor ， 但是y不是
        # loader默认会转为tensor，不用担心
        Y = torch.FloatTensor(s)
        return X,Y
dataset = DataSet()
len(dataset),dataset[1000]


(500, (tensor([0.5404, 0.4207]), tensor([0.2274])))

In [14]:
# 2.数据集加载器
# 有些英语单词是连着的，有些英语单词又用 _ 分割
loader = torch.utils.data.DataLoader(dataset = dataset,batch_size = 8, shuffle =True, drop_last = True)
# loader自动会将数据变成tensor 
len(loader),next(iter(loader))[0].shape,next(iter(loader))[1].shape

(62, torch.Size([8, 2]), torch.Size([8, 1]))

In [15]:
# 3.全连接神经网络 回归任务
class Model(torch.nn.Module):
    def __init__(self):
        #一定要写继承
        super().__init__()

        self.fc = torch.nn.Sequential(
            torch.nn.Linear(2,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,32),
            torch.nn.ReLU(),
            torch.nn.Linear(32,1),
            #回归不需要softmax  
            #torch.nn.Softmax(dim=1)
        )
    def forward(self,x):
        return self.fc(x)
    
model = Model()

model(torch.randn(8,2)).shape

torch.Size([8, 1])

In [16]:
# 4.训练
model = Model()
def train():
    optimizer = torch.optim.Adam(model.parameters(),lr=1e-4)
    #改变一下损失函数
    loss_fun = torch.nn.MSELoss()

    model.train()
    for epoch in range(100):
        for i,(x,y) in enumerate(loader):
            out = model(x)
           
            loss = loss_fun(out,y)
            #标准3布
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        if epoch%20 == 0:
            #都是训练集的结果，其实acc和loss.item()代表的含义差不多
            #acc = (out.argmax(dim=1) == y).sum().item() / len(y)
            #换一个指标，直接用loss.item()即可
            #item只能取一个元素的
            print(epoch,loss.item())
    torch.save(model,'./test1_model_regression')

#开始训练
train()

0 0.23614206910133362
20 0.0018427318427711725
40 0.0009924585465341806
60 5.855775816598907e-05
80 8.928721217671409e-05


In [17]:
# 5.测试
@torch.no_grad()
def test():
    
    #先加载模型，当然要是从头到尾跑的那也可以
    model = torch.load('./test1_model_regression')

    #进入测试模式
    model.eval()

    #获取一批数据
    x,y = next(iter(loader))
    out = model(x)
    print(out)
    print(y)
    print(y-out)
    print(torch.cat((y,out),dim=1))
    #print(out.argmax(dim=1) == y)
    #criterion = torch.nn.CrossEntropyLoss()
    #loss = criterion(out, y)
    #print(loss.item())
    #很奇怪，下面这样连这写是错误的
    #print(torch.nn.CrossEntropyLoss(out,y).item())
test()

tensor([[0.1130],
        [0.2979],
        [0.2737],
        [0.1016],
        [0.0681],
        [0.1542],
        [0.8724],
        [0.0081]])
tensor([[0.1081],
        [0.3056],
        [0.2767],
        [0.1165],
        [0.0708],
        [0.1589],
        [0.9108],
        [0.0096]])
tensor([[-0.0049],
        [ 0.0077],
        [ 0.0030],
        [ 0.0149],
        [ 0.0027],
        [ 0.0047],
        [ 0.0384],
        [ 0.0015]])
tensor([[0.1081, 0.1130],
        [0.3056, 0.2979],
        [0.2767, 0.2737],
        [0.1165, 0.1016],
        [0.0708, 0.0681],
        [0.1589, 0.1542],
        [0.9108, 0.8724],
        [0.0096, 0.0081]])
